In [ ]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [ ]:
path = r'Desktop/ipynb notebooks - practice, examples, and class files/DATA606Capstone/yelp_reviews_food_categories.csv'
# reading in merged csv as df
yelp = pd.read_csv(path)
yelp.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,...,attributes,categories,review_id,user_id,stars_y,useful,funny,cool,text,date
0,2AwhlMOMsYXJvskZKKG2GA,Tequila Sunrise,1551 N Green St,Brownsburg,IN,46112,39.867338,-86.390529,3.5,112.0,...,"{'NoiseLevel': ""u'average'"", 'BusinessParking'...","Restaurants, Mexican",djujEmxqpY5bmEJ3YaXiBg,jBzITjfkC7M99oN6a5HErQ,4.0,0.0,0.0,0.0,We'd been playing a festival in Western Indian...,2017-09-18 17:29:03
1,2AwhlMOMsYXJvskZKKG2GA,Tequila Sunrise,1551 N Green St,Brownsburg,IN,46112,39.867338,-86.390529,3.5,112.0,...,"{'NoiseLevel': ""u'average'"", 'BusinessParking'...","Restaurants, Mexican",x3LyEt-uiBFCUw65oc0y5g,R9AkporqiO758oEz_g9kxQ,3.0,0.0,0.0,0.0,"The experience happened on Tuesday, July 3rd, ...",2018-07-04 01:02:30
2,2AwhlMOMsYXJvskZKKG2GA,Tequila Sunrise,1551 N Green St,Brownsburg,IN,46112,39.867338,-86.390529,3.5,112.0,...,"{'NoiseLevel': ""u'average'"", 'BusinessParking'...","Restaurants, Mexican",zZcbhKIqxcgjWO8LJUJUaQ,-Z97B_kIy1-w3nifW1PQZg,5.0,0.0,0.0,0.0,Phe-nom-nom-nom-enal! Took my wife here today ...,2016-10-13 02:00:28
3,2AwhlMOMsYXJvskZKKG2GA,Tequila Sunrise,1551 N Green St,Brownsburg,IN,46112,39.867338,-86.390529,3.5,112.0,...,"{'NoiseLevel': ""u'average'"", 'BusinessParking'...","Restaurants, Mexican",ZoN7GbeNGFyPwwKdMEr_Mw,9m9mkjkyo4HGFUWgugewHA,5.0,0.0,0.0,0.0,"Carne Asada tacos were good, hubby had chicken...",2016-07-12 15:05:25
4,2AwhlMOMsYXJvskZKKG2GA,Tequila Sunrise,1551 N Green St,Brownsburg,IN,46112,39.867338,-86.390529,3.5,112.0,...,"{'NoiseLevel': ""u'average'"", 'BusinessParking'...","Restaurants, Mexican",NlQl6Z5UOFmAgSiMo0CbHA,KE2en6iTCiCtqVS9hq50sg,5.0,0.0,0.0,0.0,Delicious food. Very generous portions. Very...,2018-07-03 23:28:02


In [ ]:
# path = r'Desktop/ipynb notebooks - practice, examples, and class files/DATA606Capstone/yelp_academic_dataset_review.parquet'
# # reading in converted parquet as df
# yelp_review = pd.read_parquet(path)
# yelp_review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [ ]:
print(set(yelp['state']))

# drop 'states' outside US
yelp = yelp[-yelp['state'].isin(['AB', 'VI', 'XMS'])]
yelp = yelp[-yelp['state'].isnull()]
print(set(yelp['state']))

{nan, 'FL', 'AZ', 'TN', 'MO', 'PA', 'CA', 'NV', 'DE', 'LA', 'ID', 'IN', 'NJ', 'IL'}
{'FL', 'AZ', 'TN', 'MO', 'PA', 'CA', 'NV', 'DE', 'LA', 'ID', 'IN', 'NJ', 'IL'}


In [ ]:
# create a mapping of star ratings to sentiment categories
sentiment_map = {1: 'Negative', 2: 'Neutral', 3: 'Neutral', 4: 'Neutral', 5: 'Positive'}

# apply the mapping to the 'stars' column of the dataframe
yelp['sentiment'] = yelp_review['stars_y'].map(sentiment_map)

# print the resulting dataframe
yelp.head()


,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,...,categories,review_id,user_id,stars_y,useful,funny,cool,text,date,sentiment
0,2AwhlMOMsYXJvskZKKG2GA,Tequila Sunrise,1551 N Green St,Brownsburg,IN,46112,39.867338,-86.390529,3.5,112.0,...,"Restaurants, Mexican",djujEmxqpY5bmEJ3YaXiBg,jBzITjfkC7M99oN6a5HErQ,4.0,0.0,0.0,0.0,We'd been playing a festival in Western Indian...,2017-09-18 17:29:03,Neutral
1,2AwhlMOMsYXJvskZKKG2GA,Tequila Sunrise,1551 N Green St,Brownsburg,IN,46112,39.867338,-86.390529,3.5,112.0,...,"Restaurants, Mexican",x3LyEt-uiBFCUw65oc0y5g,R9AkporqiO758oEz_g9kxQ,3.0,0.0,0.0,0.0,"The experience happened on Tuesday, July 3rd, ...",2018-07-04 01:02:30,Neutral
2,2AwhlMOMsYXJvskZKKG2GA,Tequila Sunrise,1551 N Green St,Brownsburg,IN,46112,39.867338,-86.390529,3.5,112.0,...,"Restaurants, Mexican",zZcbhKIqxcgjWO8LJUJUaQ,-Z97B_kIy1-w3nifW1PQZg,5.0,0.0,0.0,0.0,Phe-nom-nom-nom-enal! Took my wife here today ...,2016-10-13 02:00:28,Positive
3,2AwhlMOMsYXJvskZKKG2GA,Tequila Sunrise,1551 N Green St,Brownsburg,IN,46112,39.867338,-86.390529,3.5,112.0,...,"Restaurants, Mexican",ZoN7GbeNGFyPwwKdMEr_Mw,9m9mkjkyo4HGFUWgugewHA,5.0,0.0,0.0,0.0,"Carne Asada tacos were good, hubby had chicken...",2016-07-12 15:05:25,Positive
4,2AwhlMOMsYXJvskZKKG2GA,Tequila Sunrise,1551 N Green St,Brownsburg,IN,46112,39.867338,-86.390529,3.5,112.0,...,"Restaurants, Mexican",NlQl6Z5UOFmAgSiMo0CbHA,KE2en6iTCiCtqVS9hq50sg,5.0,0.0,0.0,0.0,Delicious food. Very generous portions. Very...,2018-07-03 23:28:02,Positive


In [ ]:
yelp_review.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars_x', 'review_count', 'is_open',
       'attributes', 'categories', 'review_id', 'user_id', 'stars_y', 'useful',
       'funny', 'cool', 'text', 'date', 'sentiment'],
      dtype='object')

In [ ]:
# download the required NLTK resources
nltk.download('vader_lexicon')



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\}{\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
import torch

yelp['text_truncated'] = yelp['text'].apply(lambda seq: seq[:512])

print(torch.cuda.is_available())

print(torch.cuda.device_count())

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device


True
1


'cuda:0'

In [ ]:
yelp['text_truncated']

0          We'd been playing a festival in Western Indian...
1          The experience happened on Tuesday, July 3rd, ...
2          Phe-nom-nom-nom-enal! Took my wife here today ...
3          Carne Asada tacos were good, hubby had chicken...
4          Delicious food.  Very generous portions.  Very...
                                 ...                        
3584005    This is my favorite food truck! I only wish I ...
3584006    This food truck was stupid. Stupidly delicious...
3584007    Bubba never disappoints i go to his fb page an...
3584008    The truck was invited to our office for a part...
3584009    Plum Pit came and did my niece's graduation pa...
Name: text_truncated, Length: 3531254, dtype: object

In [ ]:
from transformers import pipeline

get_sentiment = pipeline('sentiment-analysis')
yelp['transformers_scores'] = yelp['text_truncated'].apply(lambda x: get_sentiment(x))


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [ ]:
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

In [ ]:
res = {}
for i, row in tqdm(yelp_review.iterrows(), total=len(yelp_review)):
    # try:
      text = row['text']
      myid = row['review_id']
      vader_result = sia.polarity_scores(text)
      vader_result_rename = {}
      for key, value in vader_result.items():
          vader_result_rename[f"vader_{key}"] = value
      roberta_result = polarity_scores_roberta(text)
      both = {**vader_result_rename, **roberta_result}
      res[myid] = both
    # except RuntimeError:
    #     print(f'Broke for id {myid}')

  0%|          | 0/6990280 [00:00<?, ?it/s]

ImportError: ignored

In [ ]:
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

In [ ]:
'''
# perform sentiment analysis on the reviews using the NLTK SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
yelp_review['sia_scores'] = yelp_review['text'].apply(lambda x: sia.polarity_scores(x))

# extract the compound scores from the sentiment analysis results
yelp_review['compound'] = yelp_review['sia_scores'].apply(lambda x: x['compound'])

# create a new column with the predicted sentiment based on the compound score
yelp_review['predicted_sentiment'] = yelp_review['compound'].apply(lambda x: 'Positive' if x > 0 else 'Negative')
'''

KeyboardInterrupt: ignored

In [ ]:
# split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(yelp['text'], yelp['sentiment'], test_size=0.2, random_state=42)

# convert the text data into a numerical format using CountVectorizer
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# train a Naive Bayes classifier on the training data
clf = MultinomialNB()
clf.fit(X_train_vect, y_train)

# predict the sentiment of the test data using the trained classifier
y_pred = clf.predict(X_test_vect)

# calculate the accuracy of the predictions
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)